load the data/
SKIP to 'np.load('newX6.npy')' to avoid having to run the pre processing code 

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.preprocessing import MinMaxScaler,StandardScaler


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.preprocessing import MinMaxScaler


dataframes_list = []
directory_path = r'/train10/train10'

for subdir in os.listdir(directory_path):
    subdir_path = os.path.join(directory_path, subdir)

    if os.path.isdir(subdir_path):  
        word_dataframes = []

        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            df = pd.read_csv(file_path)
            word_dataframes.append(df)

        dataframes_list.append(word_dataframes) # List of list of dataframes, dataframes_list[1] is the list of all data from the csv files in the addition directory


Some useful functions for the preprocessing

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.preprocessing import MinMaxScaler

def nb_changes(polarity_list):
    changes = np.diff(polarity_list)
    return np.sum(changes != 0)


import cv2
import numpy as np
def downsize(factor, tensor) : 

    resized_frames = np.empty((tensor.shape[0], 30, int(tensor.shape[2] * factor), int(tensor.shape[3] * factor),tensor.shape[4]))

    for i in range(tensor.shape[0]):
        for j in range(tensor.shape[1]):
            for k in range(tensor.shape[4]):

                resized_frame = cv2.resize(tensor[i, j, :, :,k], (0, 0), fx=factor, fy=factor)

                resized_frames[i, j, :, :,k] = resized_frame
    return resized_frames


pre process the data by splitting into frames and extracting features :
number of changes, sum of polarities, positive changes and length of the list of polarities captured in a timeframe for a given pixel.


In [ ]:
x_range = range(50, 170)
y_range = range(15, 65)
DF2 = pd.DataFrame([(x, y) for x in x_range for y in y_range], columns=['x', 'y'])
all_results = []
num_frames=30
scaler=StandardScaler()
for list_of_df in dataframes_list:
    list_results = []  # List of results for each time interval

    # Loop over each dataframe in the inner list
    for df in list_of_df:
        results = []
        med=df.loc[:,'time'].median()
        if med <600000 :
            interval_to_slice=df[(df['time'] < 1200000)]
        elif med >2400000 :
            interval_to_slice=df[(df['time'])>1800000]
        else :
            interval_to_slice=df[(df['time']>med-600000) & (df['time']<med+600000)]
        
        
        time_intervals = np.linspace(interval_to_slice['time'].to_numpy()[0], interval_to_slice['time'].to_numpy()[len(interval_to_slice)-1], num=num_frames + 1)
        for i in range(len(time_intervals) - 1):    
            start_time, end_time = time_intervals[i], time_intervals[i + 1]
            interval_data = interval_to_slice[(interval_to_slice['time'] >= start_time) & (interval_to_slice['time'] < end_time)]
            interval_data=interval_data[ (interval_data['x']>=50) & (interval_data['x']<=170) & (interval_data['y']>=15) & (interval_data['y']<=65) ]
            last_polarity_per_location =interval_data.groupby(['x', 'y'])['polarity'].sum().reset_index()
            last_pol_merged=pd.merge(DF2, last_polarity_per_location, on=['x', 'y'], how='left').fillna(0)

            count_polarity_per_location =interval_data.groupby(['x', 'y'])['polarity'].count().reset_index()
            count_pol_merged=pd.merge(DF2, count_polarity_per_location, on=['x', 'y'], how='left').fillna(0)

            grouped_data = interval_data.groupby(['x', 'y']).agg({'polarity': nb_changes}).reset_index()
            DF_merged = pd.merge(DF2, grouped_data, on=['x', 'y'], how='left')
            DF_merged['polarity'].fillna(0, inplace=True)

            matrix1=last_pol_merged.pivot(index='y', columns='x', values='polarity').values
            matrix2=count_pol_merged.pivot(index='y', columns='x', values='polarity').values
            matrix3=DF_merged.pivot(index='y', columns='x', values='polarity').values
            

            matrix3D = np.dstack((matrix1,matrix2,matrix3))
            results.append(matrix3D)
        results=np.array(results)
        list_results.append(results)
    all_results.append(np.array(list_results))
    print('one directory done')

In [ ]:
X = np.array(all_results)
X1 = X.reshape(320,30,50,120,3)
np.save('newX6resc.npy',X1) 

Load the previously preprocessed data to avoid having to run the code again

In [2]:
import numpy as np
X1=np.load('newX6resc.npy').reshape(320,30,50,120,3)

In [ ]:
def downsize(factor, tensor) : 

    resized_frames = np.empty((tensor.shape[0], 30, int(tensor.shape[2] * factor), int(tensor.shape[3] * factor),tensor.shape[4]))

    for i in range(tensor.shape[0]):
        for j in range(tensor.shape[1]):
            for k in range(tensor.shape[4]):

                resized_frame = cv2.resize(tensor[i, j, :, :,k], (0, 0), fx=factor, fy=factor)

                resized_frames[i, j, :, :,k] = resized_frame
    return resized_frames

Downsize

In [3]:
resized_frames= downsize(0.6,X1)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def update(frame):
    ax.clear()
    mean_polarity_per_location =    resized_frames[:,:,:,:,0][0][frame]

    ax.imshow(mean_polarity_per_location, cmap='Greys_r', vmin=0, vmax=1, aspect='auto', origin='lower')
    ax.invert_yaxis()
    ax.set_title(f'Polarity Heatmap - Interval {frame + 1}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

fig, ax = plt.subplots()
ani = FuncAnimation(fig, update, frames=30,interval=250, repeat=False)
plt.close()


from IPython.display import HTML
HTML(ani.to_jshtml())

Test set consisting of 3 elements from each word 

In [ ]:
##TEST SET
selected_ranges1 = [slice(3, 32)]
for i in range(35, 322, 32):
    selected_ranges1.append(slice(i, i + 29))

selected_matrix = np.concatenate([resized_frames[range_] for range_ in selected_ranges1], axis=0)

selected_ranges = []
for i in range(0, 290, 32):
    selected_ranges.append(slice(i, i + 3))
    
TEST=np.concatenate([resized_frames[range_] for range_ in selected_ranges], axis=0)
TESTlabels=np.array([i for i in range(10) for _ in range(3)])

Just to be sure, check that the test set does not contain any rows of selected_matrix (our training data) : 

In [ ]:
import numpy as np

equal_rows_indices = []

for i, test_row in enumerate(TEST):
    if any(np.array_equal(test_row, selected_row) for selected_row in selected_matrix):
        equal_rows_indices.append(i)

print("Indices of equal rows:")
print(equal_rows_indices)


LSTM model : 

In [ ]:

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Flatten,GRU, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2
from keras.layers import BatchNormalization
import tensorflow as tf
from keras.losses import SparseCategoricalCrossentropy


early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)


custom_optimizer = Adam(learning_rate=0.0001)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(selected_matrix[:,:,:,:,:].reshape(290,30,30*72*3), np.array([i for i in range(10) for _ in range(29)]), test_size=0.2, random_state=35,stratify=np.array([i for i in range(10) for _ in range(29)]))


# Create the model
model = tf.keras.models.Sequential()
model.add(BatchNormalization())
model.add(LSTM(units=64, return_sequences=True,dropout=0.1,recurrent_dropout=0.2, input_shape=(30, 30*72*3)))
model.add(Dropout(0.5))
model.add(LSTM(units=128, return_sequences=True,dropout=0.1,recurrent_dropout=0.3,activation='relu'))
#model.add(GRU(units=128, return_sequences=True, kernel_regularizer=l2(0.015),dropout=0.2,recurrent_dropout=0.25,activation='relu'))
#model.add(GRU(units=64, return_sequences=True, kernel_regularizer=l2(0.015),recurrent_dropout=0.3,activation='relu'))

model.add(Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
#model.add(Dropout(0.5))


model.add(Dense(units=10))

model.compile(optimizer=custom_optimizer, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.fit(X_train, y_train, epochs=150, batch_size=16, validation_data=(X_test, y_test), shuffle=True, callbacks=[early_stopping])

In [ ]:
Test=TEST[:,:,:,:,:]
pred=[]
raw_res=[]
for i  in range (30) : 
    result=model.predict(Test[i].reshape(1,30,30*72*3),verbose=None).argmax()
    pred.append(result)

predtrain=[]
raw_res=[]
for i  in range (len(X_test)) : 
    result=model.predict(X_test[i,:,:].reshape(1,30,30*72*3),verbose=None).argmax()
    predtrain.append(result)

t=(np.array(pred)==TESTlabels).sum()/30
t2=(np.array(predtrain)==y_test).sum()/len(X_test)
print('validation_accuracy :',t2, '\n test_accuracy :',t)

CNN/GRU

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv3D,LSTM, MaxPooling3D, Flatten, Dense,Reshape, BatchNormalization, Dropout,MaxPooling2D, Conv3D, GRU,Activation
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.losses import SparseCategoricalCrossentropy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)



y=np.array([i for i in range(10) for _ in range(29)])
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(selected_matrix[:,:,:,:,:], y, test_size=0.20, random_state=41,stratify=y)

# Build the model
custom_optimizer = Adam(learning_rate=0.0001)

model = tf.keras.models.Sequential()
model.add(BatchNormalization())
model.add(Conv3D(64, (5, 2, 2), padding='same', input_shape=(30, 30, 72,3)))
model.add(Activation('relu'))
model.add(MaxPooling3D((2,2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(128, (3, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(256, (3, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Dropout(0.1))

model.add(Reshape((3,-1)))
model.add(GRU(256, return_sequences=True, recurrent_dropout=0.1, dropout=0))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))


model.add(tf.keras.layers.Dense(10))

# Compile the model
model.compile(optimizer=custom_optimizer, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=4, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
Test=TEST[:,:,:,:,[0,1,3]]
pred=[]
raw_res=[]
for i  in range (30) : 
    result=model.predict(Test[i].reshape(1,30,30,72,3),verbose=None).argmax()
    pred.append(result)

predtrain=[]
raw_res=[]
for i  in range (len(X_test)) : 
    result=model.predict(X_test[i,:,:,:,:].reshape(1,30,30,72,3),verbose=None).argmax()
    predtrain.append(result)

t=(np.array(pred)==TESTlabels).sum()/30
t2=(np.array(predtrain)==y_test).sum()/len(X_test)
print('validation_accuracy :',t2, '\n test_accuracy :',t)

Model seems to work on our own test set, let's learn with all the data available

LSTM with all the data  :

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Flatten,GRU, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2
from keras.layers import BatchNormalization
import tensorflow as tf
from keras.losses import SparseCategoricalCrossentropy


early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)


custom_optimizer = Adam(learning_rate=0.0001)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(resized_frames[:,:,:,:,:].reshape(320,30,30*72*3), np.array([i for i in range(10) for _ in range(32)]), test_size=0.2, random_state=35,stratify=np.array([i for i in range(10) for _ in range(32)]))


# Create the model
model = tf.keras.models.Sequential()
model.add(BatchNormalization())
model.add(LSTM(units=64, return_sequences=True,dropout=0.1,recurrent_dropout=0.2, input_shape=(30, 30*72*3)))
model.add(Dropout(0.5))
model.add(LSTM(units=128, return_sequences=True,dropout=0.1,recurrent_dropout=0.3,activation='relu'))
#model.add(GRU(units=128, return_sequences=True, kernel_regularizer=l2(0.015),dropout=0.2,recurrent_dropout=0.25,activation='relu'))
#model.add(GRU(units=64, return_sequences=True, kernel_regularizer=l2(0.015),recurrent_dropout=0.3,activation='relu'))

model.add(Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(units=10))

model.compile(optimizer=custom_optimizer, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.fit(X_train, y_train, epochs=150, batch_size=16, validation_data=(X_test, y_test), shuffle=True, callbacks=[early_stopping])

In [ ]:
train_loss = model.history.history['accuracy']
val_loss = model.history.history['val_accuracy']

# Plot training loss
plt.plot(train_loss, label='Train Accuracy')
plt.plot(val_loss, label='Validation Accuracy')
plt.title('Training and Validation Accuracy LSTM')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.summary()

CNN/GRU with all the data

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv3D, MaxPooling3D,LSTM, Reshape, Flatten, Dense, BatchNormalization, Dropout,MaxPooling2D, Conv3D, GRU,Activation
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.losses import SparseCategoricalCrossentropy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)

y=np.array([i for i in range(10) for _ in range(32)])
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(resized_frames[:,:,:,:,:], y, test_size=0.2, random_state=41,stratify=y)

# Build the model
custom_optimizer = Adam(learning_rate=0.0001)

model = tf.keras.models.Sequential()
model.add(BatchNormalization())
model.add(Conv3D(64, (5, 2, 2), padding='same', input_shape=(30, 30, 72,3)))
model.add(Activation('relu'))
model.add(MaxPooling3D((2,2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(128, (3, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(256, (3, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Dropout(0.1))

model.add(Reshape((3,-1)))
model.add(GRU(256, return_sequences=True, recurrent_dropout=0.5, dropout=0))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))


model.add(tf.keras.layers.Dense(10))

# Compile the model
model.compile(optimizer=custom_optimizer, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=400, batch_size=4, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:

train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.plot(train_accuracy, label='Train Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy CNN3D/LSTM')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.summary()

In [ ]:
model.save("CNNGRUM90.keras")

Load the test set

In [ ]:
import os
dataframes_list = []
directory_path = r'/test10/test10'

for file in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)
    dataframes_list.append(df) 

preprocess it

In [ ]:
x_range = range(50, 170)
y_range = range(15, 65)
DF2 = pd.DataFrame([(x, y) for x in x_range for y in y_range], columns=['x', 'y'])
all_results = []
num_frames=30

for df in dataframes_list:
    results = []
    med=df.loc[:,'time'].median()
    if med <600000 :
        interval_to_slice=df[(df['time'] < 1200000)]
    elif med >2400000 :
        interval_to_slice=df[(df['time'])>1800000]
    else :
        interval_to_slice=df[(df['time']>med-600000) & (df['time']<med+600000)]
    
    
    time_intervals = np.linspace(interval_to_slice['time'].to_numpy()[0], interval_to_slice['time'].to_numpy()[len(interval_to_slice)-1], num=num_frames + 1)
    for i in range(len(time_intervals) - 1):   
     
        start_time, end_time = time_intervals[i], time_intervals[i + 1]
        interval_data = interval_to_slice[(interval_to_slice['time'] >= start_time) & (interval_to_slice['time'] < end_time)]
        interval_data=interval_data[ (interval_data['x']>=50) & (interval_data['x']<=170) & (interval_data['y']>=15) & (interval_data['y']<=65) ]
        last_polarity_per_location =interval_data.groupby(['x', 'y'])['polarity'].sum().reset_index()
        last_pol_merged=pd.merge(DF2, last_polarity_per_location, on=['x', 'y'], how='left').fillna(0)

        count_polarity_per_location =interval_data.groupby(['x', 'y'])['polarity'].count().reset_index()
        count_pol_merged=pd.merge(DF2, count_polarity_per_location, on=['x', 'y'], how='left').fillna(0)

        grouped_data = interval_data.groupby(['x', 'y']).agg({'polarity': nb_changes}).reset_index()
        DF_merged = pd.merge(DF2, grouped_data, on=['x', 'y'], how='left')
        DF_merged['polarity'].fillna(0, inplace=True)

        matrix1=last_pol_merged.pivot(index='y', columns='x', values='polarity').values
        matrix2=count_pol_merged.pivot(index='y', columns='x', values='polarity').values
        matrix3=DF_merged.pivot(index='y', columns='x', values='polarity').values


        matrix3D = np.dstack((matrix1,matrix2,matrix3))
        results.append(matrix3D)

    all_results.append(results)     
    print("one done")

In [ ]:
XX = np.array(all_results)
XX1 = XX.reshape(100,30,50,120,3)
np.save('newX6r.npy',XX1) ##30 frames, sum and nb of changes
XX1= downsize(0.6,XX1)


    


Get the correct labels name instead of just numbers

In [11]:
import os

directory_path = r'/train10/train10'

correct_labels = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]


In [6]:
XX3=np.load('newX6r.npy')

In [7]:
XX3= downsize(0.6,XX3)

In [5]:
import tensorflow as tf
model=tf.keras.models.load_model('CNNGRUM90.keras')

Predict 

In [13]:
pred=[]
for i  in range (100) : 
    result=model.predict(XX3[i].reshape(1,30,30,72,3),verbose=None).argmax()
    pred.append(correct_labels[result])

In [14]:
import os


directory_path = r'/test10/test10'

file_names = os.listdir(directory_path)

numbers = [int(file_name.split('.')[0]) for file_name in file_names]



In [15]:
id = numbers

data_list = list(zip(id, pred))
import csv
file_path = 'sampleSubmission.csv'
with open(file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['id', 'label'])  # Header
    csv_writer.writerows(data_list)

In [16]:
id = [i for i in range(100)]

data_list = list(zip(id, pred))
import csv
file_path = 'sampleSubmission.csv'
with open(file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['id', 'label'])  # Header
    csv_writer.writerows(data_list)